# RLS-CLS

## Column Level Security

### Create schema in sales_catalog

In [0]:
USE CATALOG sales_catalog;
DROP SCHEMA IF EXISTS secure_schema CASCADE;
CREATE SCHEMA secure_schema;
USE SCHEMA secure_schema;

### Add Masking Policies to Columns

In [0]:
-- Create masking policy for email
CREATE OR REPLACE FUNCTION email_mask(email STRING)
RETURN CASE
  WHEN is_account_group_member('finance_team') THEN email
  ELSE CONCAT('***@', substring_index(email, '@', -1))
END;

-- Create masking policy for phone
CREATE OR REPLACE FUNCTION phone_mask(phone STRING)
RETURN CASE
  WHEN is_account_group_member('finance_team') THEN phone
  ELSE CONCAT('******', RIGHT(phone, 4))
END;

### Create customer with sensitive data

In [0]:
CREATE OR REPLACE TABLE customer (
  customer_id INT,
  customer_name STRING,
  customer_email STRING MASK email_mask,
  customer_phone STRING MASK phone_mask,
  credit_card STRING,
  region STRING
);

-- Insert sample records
INSERT INTO customer VALUES
(101, 'John Doe',   'john@example.com',  '9876543210', '4567-3453-5567-2302', 'North'),
(102, 'Alice Smith','alice@example.com', '8765432109', '1234-5678-9012-3456', 'South'),
(103, 'Mark Lee',   'mark@example.com',  '7654321098', '9999-8888-7777-6666', 'North'),
(104, 'Lucy Ray',   'lucy@example.com',  '6543210987', '4321-4321-4321-4321', 'East');


In [0]:
SELECT * FROM customer where customer_email = 'john@example.com'

In [0]:
DESCRIBE EXTENDED customer

### Create Masking Policy for Credit Card

In [0]:
CREATE OR REPLACE FUNCTION credit_card_mask(ccn STRING)
RETURN CASE
  WHEN is_account_group_member('finance_team') THEN ccn
  ELSE CONCAT('xxxx-xxxx-xxxx-', RIGHT(ccn, 4))
END;

### Alter the Table to Add Credit Card Column

In [0]:
ALTER TABLE customer
ALTER COLUMN credit_card
SET MASK credit_card_mask;

In [0]:
SELECT customer_id, customer_name, credit_card FROM customer;

## Row Level Security

### Create a Row Filter Function

In [0]:
CREATE OR REPLACE FUNCTION region_filter(region STRING)
RETURN is_account_group_member('regions') or is_account_group_member('admins');

### Create sample data

In [0]:
CREATE OR REPLACE TABLE sales_data (
  sale_id INT,
  emp_name STRING,
  region STRING,
  product STRING,
  quantity INT,
  amount DOUBLE
)
WITH ROW FILTER region_filter ON (region);

INSERT INTO sales_data VALUES
(1, 'Jones',    'North', 'Laptop',   2, 2000.0),
(2, 'Smith',    'South', 'Phone',    1,  800.0),
(3, 'Alice',    'East',  'Tablet',   3, 1500.0),
(4, 'Bob',      'West',  'Monitor',  2,  600.0),
(5, 'Charlie',  'North', 'Printer',  1,  300.0),
(6, 'Diana',    'South', 'Mouse',    4,  200.0),
(7, 'Eve',      'East',  'Laptop',   1, 1000.0),
(8, 'Frank',    'West',  'Phone',    2, 1600.0),
(9, 'Grace',    'North', 'Tablet',   2, 1200.0),
(10,'Hank',     'South', 'Keyboard', 3,  450.0);

In [0]:
SELECT * FROM sales_data;

### Create a Dynamic Region Filter Function

In [0]:
CREATE OR REPLACE FUNCTION dynamic_region_rls(region STRING)
RETURN is_account_group_member(CONCAT(lower(region), '_analyst'));

In [0]:
select CONCAT(lower('North'), '_analyst')

In [0]:
SELECT current_user()

### Apply the Filter via ALTER TABLE

In [0]:
ALTER TABLE sales_data
SET ROW FILTER dynamic_region_rls ON (region);

In [0]:
SELECT * FROM sales_data;

In [0]:
-- Remove an Existing Row Filter (if needed)

-- ALTER TABLE sales_data
-- UNSET ROW FILTER;

## Create a dynamic view

In [0]:
CREATE OR REPLACE VIEW customer_redacted AS
SELECT
  customer_id,
  CASE
    WHEN is_account_group_member('auditors') THEN customer_email
    ELSE 'REDACTED'
  END AS customer_email,
  customer_name,
  region
FROM customer;

In [0]:
SELECT * FROM customer_redacted where customer_email = 'john@example.com'

In [0]:
CREATE OR REPLACE VIEW sales_redacted AS
SELECT
  emp_name,
  region,
  product,
  amount
FROM sales_data
WHERE
  is_account_group_member('managers') OR amount <= 1000;

In [0]:
SELECT * FROM sales_redacted